In [ ]:
%config IPCompleter.greedy = True

In [ ]:
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import matplotlib as plt
%matplotlib inline

import helper

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = datasets.FashionMNIST('FashionMNIST_data/', download = True, train = True, transform = transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True)

In [ ]:
test_set = datasets.FashionMNIST('FashionMNIST_data/', download = True, train = False, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True)

In [ ]:
class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p):
        super().__init__()
        
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
            
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes ])
        
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        self.dropout = nn.Dropout(p = drop_p)
        
        
    def forward(self, x):
        
        for linear in self.hidden_layers:
            x = F.relu(linear(x))
            x = self.dropout(x)
        
        x = self.output(x)
        
        return F.log_softmax(x, dim = 1)

In [ ]:
model = Network(784, 10, [516, 256], 0.5)
print(model)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.NLLLoss()

In [ ]:
def validation(model, test_loader, criterion):
    accuracy = 0
    test_loss = 0
    for images, labels in test_loader:
        #images, labels = images.to('cuda'), labels.to('cuda')
        images = images.resize_(images.size()[0], 784)
        
        output = model.forward(images)
        ps = torch.exp(output)
        
        test_loss += criterion(output, labels).item()
        
        equality = (labels.data == ps.max(dim = 1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [ ]:
epochs = 3
steps = 0
print_every = 40
#model.to('cuda')
for e in range(epochs):
    running_loss = 0
    model.train()
    for images, labels in train_loader:
        steps +=1
        
        #images, labels = images.to('cuda'), labels.to('cuda')
        
        images.resize_(images.shape[0], 784)
        
        optimizer.zero_grad()
        
        #output = model(images)
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            
            with torch.no_grad():
                test_loss, accuracy = validation(model, test_loader, criterion)
                print(
                    'running_loss : {:.3f}'.format(running_loss/print_every),
                    'test_loss : {:.3f}'.format(test_loss/len(test_loader)),
                    'accuracy {:.3f}'.format(accuracy/len(test_loader))
                )
                
            running_loss = 0
            model.train()

In [ ]:
checkpoint = {
                'input_size':784,
                'output_size':10,
                'hidden_layers':[each.out_features for each in model.hidden_layers],
                'state_dict':model.state_dict()
            }

In [ ]:
torch.save(checkpoint, 'checkpoint.pth')